In [ ]:
import pandas as pd

ports = pd.read_csv("./ports.csv", sep='\t')
tracking = pd.read_csv("./tracking.csv", sep="\t")
tracking

,vessel,datetime,lat,long,heading,speed,draft
0,1,4/21/2019 12:49,-69.844458,13.256915,268.0,0.1,7.800
1,2,5/27/2019 7:15,-56.516713,-67.264773,118.0,16.1,9.500
2,2,5/15/2019 4:18,-56.507373,-66.961833,246.0,13.2,11.199
3,3,9/9/2019 2:43,-56.438527,-67.176253,258.0,17.8,11.400
4,2,5/27/2019 8:50,-56.383307,-66.600653,66.0,10.8,9.500
...,...,...,...,...,...,...,...
802663,62,10/15/2019 3:28,73.868687,70.932767,83.0,7.8,11.400
802664,62,9/24/2019 20:38,73.868730,70.630372,89.0,11.6,9.200
802665,62,9/28/2019 2:54,73.870265,70.716972,272.0,15.9,9.300
802666,62,9/28/2019 2:55,73.870287,70.702728,272.0,15.9,9.300


In [ ]:
tracking = tracking.loc[tracking['speed'] == 0]
tracking['datetime'] = pd.to_datetime(tracking['datetime'], infer_datetime_format=True)
tracking = tracking.sort_values(by="datetime")

In [ ]:
ports_tol = pd.read_csv('./ports_tol.csv')
tol = ports_tol.set_index('port').to_dict()['variance']

In [ ]:
len(tol)

42

In [ ]:
import sys

output = pd.DataFrame(columns=['vessel','datetime','port','lat','long'])

for i, rowV in tracking.iterrows(): 

  minDist = sys.maxsize
  vessel = rowV['vessel']
  lat = rowV['lat']
  long = rowV['long']
  date = rowV['datetime']

  for j, rowP in ports.iterrows():
    if rowP['port'] in tol.keys():
      t = tol[rowP['port']]
    else:
      t = 0.03

    if abs(long - rowP['long']) + abs(lat - rowP['lat']) <= t:
      if abs(long - rowP['long']) + abs(lat - rowP['lat']) < minDist:
        minDist = abs(long - rowP['long']) + abs(lat - rowP['lat'])
        port = rowP['port']
        data=[vessel, date, port, lat, long]
        output.loc[i] = (data)

In [ ]:
output = output.sort_values(by=['vessel','datetime'])

In [ ]:
output.to_csv("output.csv", index=False)


In [ ]:
voyages = pd.DataFrame(columns=['vessel', 'begin_date', 'end_date', 'begin_port_id', 'end_port_id'])
vessel = output['vessel'].iloc[0]
begin_port_id = output['port'].iloc[0]
begin_date = output['datetime'].iloc[0]

for i in range(len(output) - 1):
  if output['vessel'].iloc[i+1] != output['vessel'].iloc[i]:
    vessel = output['vessel'].iloc[i+1]

  if output['port'].iloc[i+1] != output['port'].iloc[i] and output['datetime'].iloc[i] < output['datetime'].iloc[i+1]: 
    begin_date = output['datetime'].iloc[i]
    begin_port_id = output['port'].iloc[i]
    end_date = output['datetime'].iloc[i+1]
    end_port_id = output['port'].iloc[i+1]
    voyages.loc[i] = [vessel, begin_date, end_date, begin_port_id, end_port_id]
    
    

  

  

In [ ]:
voyages = voyages.sort_values(by=['vessel','begin_date'])

In [ ]:
voyages.to_csv("voyages.csv", index=False)